In [346]:
import pandas as pd
import re
import numpy as np
import math

In [384]:
json = pd.read_json("data.json")

In [390]:
df = pd.DataFrame(json.data.values.tolist())
df.head()

,title,work_experience,region,salary
0,Backend разработчик со знанием Python,3–6 лет,Алматы,от 2 500 до 3 000 USD на руки
1,Backend разработчик со знанием Python,3–6 лет,Баку,от 2 500 до 3 000 USD на руки
2,Backend разработчик со знанием Python,3–6 лет,Бишкек,от 2 500 до 3 000 USD на руки
3,Backend разработчик,3–6 лет,Минск,от 2 500 до 3 000 USD на руки
4,Backend разработчик со знанием Python,3–6 лет,Ташкент,от 2 500 до 3 000 USD на руки


In [349]:
df['salary'] = df['salary'].fillna(0)

In [350]:
def get_current_salary(salary):
    usd = False
    if salary == 0:
        return 0
    if "USD" in salary:
        usd = True
    m = re.findall(r'\d+', salary)
    if len(m)>0:
        s = int(str(m[0])+str(m[1]))
        if usd:
            return s*60
        return s
    return 0

In [351]:
def get_type_direction(title):
    dev =["backend", "python-разработчик","программист", 
          "python разработчик","python developer","python developer","Django"]
    ml = ["машин", "ml"]
    ds = ["data scientist"]
    de = ["data engineer"]
    devops = ["devops"]
    analyst = ["analyst"]
    qa = ["qa"]
    lt = title.lower()
    if "engenering" in lt:
        return "Software Engenering"
    elif any(s.lower() in lt for s in dev):
        return "developer"
    elif any(s.lower() in lt for s in qa):
        return "QA"
    elif any(s.lower() in lt for s in ml):
        return "ML"
    elif any(s.lower() in lt for s in de):
        return "DE"
    elif any(s.lower() in lt for s in ds):
        return "DS"
    elif any(s.lower() in lt for s in devops):
        return "DevOps"
    elif any(s.lower() in lt for s in analyst):
        return "Analyst"
    else:
        return "etc"

In [352]:
def get_work_experience(work_experience):
    if work_experience == 'не требуется':
        return 0
    if "более" in work_experience:
        return int(re.findall(r'\d+', work_experience)[0])
    return int(work_experience.split("–")[0])

In [353]:
df["direction"] = df.apply(lambda x: get_type_direction(x.title), axis=1)
df["salary"] = df.apply(lambda x: get_current_salary(x.salary), axis=1)
df["work_experience"] = df.apply(lambda x: get_work_experience(x.work_experience), axis=1)

In [354]:
df = df.replace(0, np.NaN)

### Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)

In [355]:
direction_group = df.groupby(by="direction")

### Какая средняя и медианная зарплата по группам вакансий?

In [356]:
print(round(direction_group.salary.mean()))
print(round(direction_group.salary.median()))

direction
Analyst       36500.0
DE           250000.0
DS           100000.0
DevOps       200833.0
ML           150000.0
QA           162000.0
developer    140185.0
etc          132000.0
Name: salary, dtype: float64
direction
Analyst       36500.0
DE           250000.0
DS           100000.0
DevOps       215000.0
ML           150000.0
QA           100000.0
developer    150000.0
etc          100000.0
Name: salary, dtype: float64


### Какая средняя и медианная зарплата по каждому региону?

In [357]:
region_group=df.groupby(by="region")

In [358]:
print(region_group.salary.mean())
print(region_group.salary.median())

region
Алматы         225000.0
Армения             NaN
Архангельск     50000.0
Баку           150000.0
Балашиха        60000.0
                 ...   
Чебоксары           NaN
Челябинск      100000.0
Череповец           NaN
Черногория          NaN
Ярославль      100000.0
Name: salary, Length: 72, dtype: float64
region
Алматы         225000.0
Армения             NaN
Архангельск     50000.0
Баку           150000.0
Балашиха        60000.0
                 ...   
Чебоксары           NaN
Челябинск      100000.0
Череповец           NaN
Черногория          NaN
Ярославль      100000.0
Name: salary, Length: 72, dtype: float64


### Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?

In [359]:
mean_salary = direction_group.salary.mean()
max_salary = mean_salary.max()
mean_salary.describe
mean_salary.loc[mean_salary == max_salary]

direction
DE    250000.0
Name: salary, dtype: float64

### Какое процентное соотношение каждого региона по вакансиям от всех вакансий?

In [360]:
total_vacancy = df.title.count()
one_procent = total_vacancy/100
reg_stat = round(region_group.title.count()/one_procent,2)

In [361]:
reg_stat.sort_values(ascending=False)

region
Москва             24.00
Санкт-Петербург     7.25
Минск               2.12
Алматы              2.00
Нижний Новгород     1.88
                   ...  
Курган              0.12
Омск                0.12
Оренбург            0.12
Пенза               0.12
Королев             0.12
Name: title, Length: 72, dtype: float64

### Какая корреляция уровня опыта от зарплаты?

In [363]:
dfcorr = pd.DataFrame(df, columns=["work_experience","salary"])

In [367]:
dfcorr.corr().loc["work_experience","salary"]

0.21594245890514074

### Сколько должностей в наборе данных?

In [378]:
df["title"].nunique()

590

### Какие 10 наиболее часто встречающихся должностей?

In [379]:
df_count = df.groupby(['title'])['title'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)

In [382]:
df_count[:10]

,title,Count
184,Python Developer,19
511,Разработчик Python,17
466,Программист Python,14
202,Python developer,12
415,Инженер-программист,9
396,Ведущий сетевой инженер в ЦОД,9
327,Senior/Lead Python-разработчик,6
231,Python-разработчик,6
61,DevOps Engineer,6
140,Middle Python Developer,5


In [383]:
df.tail()

,title,work_experience,region,salary,direction
795,Заместитель начальника отдела разработки инфор...,3.0,NaN,100000.0,etc
796,Разработчик машинного обучения в Алису,3.0,"Москва, Парк культуры",NaN,ML
797,Инженер по тестированию / QA engineer,1.0,Москва,NaN,QA
798,Data analyst (CV),3.0,Алматы,NaN,Analyst
799,Администратор сред,1.0,Москва,NaN,etc
